In [ ]:
#!pip install pytorch_lightning
#!pip install torchsummaryX
!pip install webdataset
# !pip install datasets
# !pip install wandb
#!pip install -r MedicalZooPytorch/installation/requirements.txt
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/McMasterAI/Radiology-and-AI.git   #--branch augmentation 
!git clone https://github.com/jcreinhold/intensity-normalization.git
! python intensity-normalization/setup.py install
!pip install scikit-fuzzy

     |████████████████████████████████| 133kB 14.6MB/s 
Cloning into 'Radiology-and-AI'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 312 (delta 92), reused 98 (delta 44), pack-reused 160
Receiving objects: 100% (312/312), 25.81 MiB | 23.06 MiB/s, done.
Resolving deltas: 100% (170/170), done.
Cloning into 'intensity-normalization'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 1216 (delta 111), reused 85 (delta 12), pack-reused 1032
Receiving objects: 100% (1216/1216), 1.03 MiB | 3.33 MiB/s, done.
Resolving deltas: 100% (846/846), done.
Traceback (most recent call last):
  File "intensity-normalization/setup.py", line 39, in <module>
    with open('README.md', encoding="utf-8") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'README.md'
     |████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/MacAI

/content/drive/MyDrive/MacAI


In [ ]:
import sys
sys.path.append('./Radiology-and-AI/Radiology_and_AI')
sys.path.append('./intensity-normalization')
import os
import torch
import numpy as np
import webdataset as wds

In [ ]:
import intensity_normalization

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from io import BytesIO
from nibabel import FileHolder, Nifti1Image
import torch
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from scipy.ndimage.filters import gaussian_filter
from time import time

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
train_dataset = wds.Dataset("macai_datasets/brats/train/brats_train.tar.gz")
eval_dataset = wds.Dataset("macai_datasets/brats/validation/brats_validation.tar.gz")

In [ ]:
def np_img_collator(batch):
    bytes_data_list = [list(batch[i].items())[1][1] for i in range(5)]
    bytes_data_keys = [list(batch[i].items())[0][1].split('_')[-1] for i in range(5)]
    bytes_data_dict = dict(zip(bytes_data_keys,bytes_data_list))

    bb = BytesIO(bytes_data_dict['flair'])
    fh = FileHolder(fileobj=bb)
    f_flair = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_dict['seg'])
    fh = FileHolder(fileobj=bb)
    f_seg = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_dict['t1'])
    fh = FileHolder(fileobj=bb)
    f_t1 = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_dict['t1ce'])
    fh = FileHolder(fileobj=bb)
    f_t1ce=Nifti1Image.from_file_map({'header':fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_dict['t2'])
    fh = FileHolder(fileobj=bb)
    f_t2 =Nifti1Image.from_file_map({'header':fh, 'image':fh}).get_fdata()

    padding = [(0, 0), (0, 0), (0, 0)]# last (2,3)
    f_flair = np.expand_dims(np.pad(f_flair, padding), axis=0)
    f_t1 = np.expand_dims(np.pad(f_t1, padding), axis=0)
    f_t2 = np.expand_dims(np.pad(f_t2, padding), axis=0)
    f_t1ce = np.expand_dims(np.pad(f_t1ce, padding), axis=0)

    f_seg = np.pad(f_seg, padding)
    concat = np.concatenate([f_t1, f_t1ce, f_t2, f_flair], axis=0)
    f_seg = np.expand_dims(f_seg, axis=0)

    return ([concat, f_seg])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5,collate_fn=np_img_collator)

In [ ]:
def nyul_train_dataloader(dataloader, n_imgs = 4, i_min=1, i_max=99, i_s_min=1, i_s_max=100, l_percentile=10, u_percentile=90, step=20):
    """
    determine the standard scale for the set of images
    Args:
        img_fns (list): set of NifTI MR image paths which are to be normalized
        mask_fns (list): set of corresponding masks (if not provided, estimated)
        i_min (float): minimum percentile to consider in the images
        i_max (float): maximum percentile to consider in the images
        i_s_min (float): minimum percentile on the standard scale
        i_s_max (float): maximum percentile on the standard scale
        l_percentile (int): middle percentile lower bound (e.g., for deciles 10)
        u_percentile (int): middle percentile upper bound (e.g., for deciles 90)
        step (int): step for middle percentiles (e.g., for deciles 10)
    Returns:
        standard_scale (np.ndarray): average landmark intensity for images
        percs (np.ndarray): array of all percentiles used
    """
    percss = [np.concatenate(([i_min], np.arange(l_percentile, u_percentile+1, step), [i_max])) for _ in range(n_imgs)]
    standard_scales = [np.zeros(len(percss[0])) for _ in range(n_imgs)]

    iteration = 1
    for all_img, seg_data in dataloader:
      print(iteration)
    
     # print(seg_data.shape)
      mask_data = seg_data
      mask_data[seg_data ==0] = 1
      mask_data = np.squeeze(mask_data, axis=0)

      #mask_data[mask_data==2] = 0 # ignore edema

      for i in range(n_imgs):
        img_data = all_img[i]

        masked = img_data[mask_data > 0]
        landmarks = intensity_normalization.normalize.nyul.get_landmarks(masked, percss[i])
        min_p = np.percentile(masked, i_min)
        max_p = np.percentile(masked, i_max)
        f = interp1d([min_p, max_p], [i_s_min, i_s_max])
        landmarks = np.array(f(landmarks))
        
        standard_scales[i] += landmarks
      iteration += 1

    standard_scales = [scale / iteration for scale in standard_scales]
    return standard_scales, percss

In [ ]:
standard_scales, percss = nyul_train_dataloader(train_dataloader)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
def dataloader_hist_norm(img_data, landmark_percs, standard_scale, seg_data):
    """
    do the Nyul and Udupa histogram normalization routine with a given set of learned landmarks
    Args:
        img (nibabel.nifti1.Nifti1Image): image on which to find landmarks
        landmark_percs (np.ndarray): corresponding landmark points of standard scale
        standard_scale (np.ndarray): landmarks on the standard scale
        mask (nibabel.nifti1.Nifti1Image): foreground mask for img
    Returns:
        normalized (nibabel.nifti1.Nifti1Image): normalized image
    """
    mask_data = seg_data
    mask_data[seg_data ==0] = 1
    mask_data = np.squeeze(mask_data, axis=0)

    masked = img_data[mask_data > 0]
    landmarks = intensity_normalization.normalize.nyul.get_landmarks(masked, landmark_percs)
    f = interp1d(landmarks, standard_scale, fill_value='extrapolate')
    normed = f(img_data)

    z = img_data
    z[img_data > 0] = normed[img_data > 0]
    return z #normed

In [ ]:
for all_img, seg_data in train_dataloader:
  for i, this_img in enumerate(all_img):
    if i == 0:
      transformed_img = dataloader_hist_norm(this_img, percss[i], standard_scales[i], seg_data)
      transformed_img = transformed_img[transformed_img>0]
      plt.hist(np.ravel(transformed_img), bins=30)
      plt.xlim(0, 150)
      plt.show()
     # plt.hist(np.ravel(this_img))
     # plt.show()

Output hidden; open in https://colab.research.google.com to view.